# Detrending Periodic Motions in Video

The goal of this notebook is to demonstrate a proof of concept for cleaning up periodic motions in videos.  If the input is a periodically oscillating blob with an additive random walk, the network will try to remove the random walk and only be left with the periodic motion

In [ ]:
%load_ext autoreload
%autoreload 2
from persim import plot_diagrams
from takenslayers import *
from takenspipelines import *
from videoutils import *
import matplotlib.pyplot as plt

import math

import numpy as np
from scipy.signal import convolve
from skimage.transform import resize

import torch
from torch.autograd import Variable
from torch import nn
from torch.autograd import Variable
from torch.nn import functional as F
import torchvision.transforms as T

torch.manual_seed(2)

In [ ]:
frames = load_video("Videos/face.mp4", make_rgb=True)
#frames = frames[:, :, :, 100::]
win = 40

X = np.moveaxis(frames, [0, 1, 2, 3], [2, 3, 1, 0])
X = np.array(X, dtype=float)/255
X = X[:, :, 0:512, 0:512]
#XMu = np.mean(X, axis=0)
#X -= XMu[None, :, :, :]
#XMax = np.max(np.abs(X))
#X /= XMax


#XOut = X/np.max(X)
#XOut = np.array(255*X, dtype=np.uint8)
#XOut = np.moveaxis(XOut, [2, 3, 1, 0], [0, 1, 2, 3])
#save_video("MeanSubtract.avi", XOut, is_rgb=True)


#XR = np.reshape(X, (X.shape[0], X.shape[1], X.shape[2]*X.shape[3]))
#XR = np.moveaxis(XR, [0, 1, 2], [2, 0, 1])
#XR = np.reshape(XR, (XR.shape[0], XR.shape[1]*XR.shape[2]))
#print(np.std(XR, axis=1))
print("X.shape", X.shape)
data = torch.from_numpy(X).float()

bl = BandpassLayer(0.5, 3, 30)
#data = bl(data)
print("data.shape", data.shape)

batch_norm = torch.nn.BatchNorm2d(3)
vid_dist = SlidingVideoDistanceMatrixLayer(win, 'cpu')
rips = RipsPersistenceDistance([1])
D = vid_dist(batch_norm(bl(data)))
dgm = rips(D)[0]

plt.figure(figsize=(12, 6))
plt.subplot(121)
plt.imshow(D.detach().numpy())
plt.colorbar()
plt.title("Original SSM")
plt.subplot(122)
dgm = dgm.detach().numpy()
if dgm.size > 0:
    plot_diagrams(dgm, labels=["H1"])
plt.title("Original Persistence Diagram")

In [ ]:
def get_device():
    if torch.cuda.is_available():
        device = 'cuda:0'
    else:
        device = 'cpu'
    return 'cpu'#device

print("win = ", win)
block_win = 80
block_hop = 4
device = get_device()
print(device)
data = data.to(device)
autoencoder = BlockRipsRegularizedAutoencoderCNN(device, data, depth=3, dim=100, win=win, block_win=block_win,
                                            block_hop=block_hop, lam_block=0.0005, lam_var = 0, lr=0.001, last_layer=nn.Sigmoid)
#autoencoder = RipsRegularizedAutoencoderCNN(device, data, depth=3, dim=100, win=win, lam=0.001, lr=0.001, last_layer=nn.Sigmoid)
autoencoder = autoencoder.to(device)

y = autoencoder.train_epochs(300, plot_interval=1)

In [ ]:
amp_fac=4
res = np.array(y.cpu().detach().numpy())
mu = np.mean(res, axis=0)
res -= mu
res *= amp_fac
res += mu
#res *= XMax*amp_fac
#res += XMu[None, :, :, :]
frames_res = np.moveaxis(res, (2, 3, 0, 1), (0, 1, 3, 2))

plt.subplot(211)
plt.plot(frames[64, 64, 0, :]/255)
plt.subplot(212)
plt.plot(res[:, 0, 64, 64])
#print(XMu.shape)

In [ ]:
frames_res = np.array(frames_res*255, dtype=np.uint8)
frames_res[frames_res < 0] = 0
frames_res[frames_res > 255] = 255
save_video("result.avi", frames_res, is_rgb=True)